In [1]:
from db_connect import db_get_df
from segment_ranking.rank_segments import get_most_similar_documents
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from transformers import BertModel, BertTokenizer

In [3]:
df = db_get_df(table="transcript_segments")
df.dtypes

filename           object
segment_text       object
start             float64
end               float64
segment_id          int64
embedding_json     object
dtype: object

Wir verwenden hier das Deutsche model, um die deutschen Transkripte optimal zu encodieren.

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
model = BertModel.from_pretrained('bert-base-german-cased', output_hidden_states = True)

In [5]:
df["tokens"] = [tokenizer.tokenize(text) for text in df["segment_text"]]

In [7]:
print(df["tokens"].sample(2).to_markdown())

|      | tokens                                                                                                                                                                    |
|-----:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1538 | ['unver', '##brü', '##ch', '##liche', 'Treue', '.', 'Die', 'Natur', 'hat', 'alles', 'im', 'Angebot', '.', 'Und', 'der', 'Mensch', '?', 'Zumindest', 'nimmt', 'er', 'für'] |
| 1058 | ['über', 'mehrere', 'Jahre', 'insgesamt', 'acht', 'Gebäude', 'frei', ',', 'in', 'denen', 'sie', 'Reste', 'einer', 'Schm', '##iede', ',', 'einer', 'Feuer', '##stelle']    |
